In [ ]:
import seaborn as sns
import numpy as np
from baynes.model_utils import get_model
from baynes.analysis import *
from baynes.probability import hdi
import pandas as pd

import cmdstanpy
import logging
cmdstanpy.utils.get_logger().setLevel(logging.ERROR)

# Prior sensitivity analysis for the Poisson model
### Generate the data

In [ ]:
N=1000
lambda_true = 6.3
events = np.random.poisson(lambda_true, N)

data_mean = np.mean(events)
data_sd = np.std(events)
print('- mean of data: ', data_mean)
print('- sd of data: ', data_sd)
bins = np.arange(min(events), max(events))
sns.histplot(events, bins=bins)
sns.despine()

data = {'N': len(events), 'y': events, 'alpha': 5, 'beta': 1, 'prior': 0}

### Compile and print the STAN model


In [ ]:
model = get_model('poisson.stan')
print(model.code())

`baynes` allows to repeat the fit for multiple data keys and values using multithreading

In [ ]:
from baynes import sensitivity_sweep
df = sensitivity_sweep(model, data, {'alpha': [1, 5, 10, 20, 40, 60]}, ['lambda'])
sns.lineplot(df, x='alpha', y='lambda', errorbar=hdi)

The sweep over different data must be coded explicitly for more complex cases

In [ ]:
df=pd.DataFrame()
prior_var=10
post=[]
mus= np.linspace(1, 51, 10)

Ns=[50, 100, 200, 500]
for N in Ns:
    evs = events[:N]
    data = {'N': len(evs), 'y': evs, 'alpha': 5, 'beta': 1, 'prior': 0}
    dN = pd.DataFrame()
    for mu in mus:
        data['beta'] = mu/prior_var
        data['alpha'] = mu**2/prior_var

        fit = model.sample(data,
                           chains=4,
                           iter_warmup=200,
                           iter_sampling=300,
                           save_warmup=False,
                           show_progress=False)
        dfit = fit.draws_pd(['lambda'])
        dfit['prior mean'] = mu
        dN = pd.concat([dN, dfit])
    dN['N'] = N
    df = pd.concat([df, dN])

In [ ]:
ax = sns.lineplot(df, x='prior mean', y='lambda', hue='N', errorbar=hdi, zorder=0, err_kws={'alpha':0.4, 'lw':0}, style='N')
ax.legend(bbox_to_anchor=(1.1, 0.7), facecolor='white', edgecolor='white', title='N data')